## Example: quartz sand dissolution

**Authors: G. Dan Miron, Georg Kosakowski**

## Import python packages

In [ ]:
import numpy as np
import xgems as xg

## Setting global variables, paths

In [ ]:
# file name for system definitions
xg_system_filename='gems_files/mortar-dat.lst'

xgEngine = xg.ChemicalEngine(xg_system_filename)

## Definning the lists of indexes for elements, substance
- hint these can be kept in a separate python file an used in other projects

In [ ]:
# copy from leaching example, add 'Quartz': 'quartz',
# lists of elements and phases
element_symbols = ['Al', 'C', 'Ca', 'Cl', 'Fe', 'H', 'K', 'Mg', 'Na', 'O', 'S', 'Si', 'Zz']
phase_names = {
    'Quartz': 'quartz',
    'CSHQ': 'CSH',
    'ettringite': 'ettrignite',
    'SO4_CO3_AFt': 'ettrignite',
    'CO3_SO4_AFt': 'ettrignite',
    'thaumasite': 'thaumasite',
    'SO4_OH_AFm': 'monosulphate',
    'OH_SO4_AFm': 'monosulphate',
    'C4AcH11': 'monocarbonate',
    'OH-hydrotalcite': 'hydrotalcite',
    'Kuzels': 'Kuzel_s',
    'Friedels': 'Friedel_s',
    'straetlingite': 'straetlingite',
    'Calcite': 'calcite',
    'MSH': 'MSH',
    'Gypsum': 'gypsum',
    'Portlandite': 'portlandite',
    'Natrolite': 'zeolites',
    'ZeoliteX': 'zeolites',
    'ZeoliteY': 'zeolites',
    'ZeoliteP': 'zeolites',
    'Chabazite': 'zeolites',
    'C3(AF)S0.84H': 'hydrogarnet',
    'C3FS1.34H3.32': 'hydrogarnet',
    'Ferrihydrite-mc': 'ferryhidrite',
    'Al(OH)3mic': 'Al(OH)3(mic)',
    'aq_gen': 'aqueous',
}

# dictionary of element names and index
element_indexes = {symbol: xgEngine.indexElement(symbol) for symbol in element_symbols}
# dictionary of phase names and index
phase_indexes = {name: xgEngine.indexPhase(name) for name in phase_names}

In [ ]:
# copy from leaching example, add Al, Mg
# Lists of what we will output 
# properties change cycle with time, add rate
output_aqueous_elements = ['Ca', 'Si','Al','Mg', 'S', 'C']
output_properties = ['time', 'pH', 'gems_code','rate', 'Ca/Si in CSH']

In [ ]:
# copy from leaching example
# initalize the output containers, also dictionaries
# imagine these like an empty table with column names and values in rows

aqueous_table = {element: [] for element in output_aqueous_elements}

# Since we want the group names rather than individual phases,  
# we loop through the dictionary values instead of the keys.
solids_volume_table = {phase: [] for phase in phase_names.values()}

properties_table = {prop: [] for prop in output_properties}


In [ ]:
# copy from leaching example
# Finally, we want to create a dictionary where the keys are group names (outputs),  
# and the values are lists of indexes for the phases that belong to each group. (i.e., more phases belong to ettringite)

grouped_output_phase_indexes = {}
for phase, name in phase_names.items():
    grouped_output_phase_indexes.setdefault(name, []).append(phase_indexes[phase])
#grouped_output_phase_indexes

## Defining the kinetic function

In [ ]:
# writting a function that calculates the dissolution rate 

# A Python function is a reusable block of code that performs a specific task.  
# You define it using `def`, give it a name, optionally specify input parameters, and use `return` to output a result.  

#This function calculates the quartz dissolution rate based on temperature (T), surface area (S), saturation state (omega), and activities of H⁺ (aHplus) and OH⁻ (aOHminus).  
#It computes contributions from neutral, acid, and base reaction mechanisms using given constants and returns their sum as the overall dissolution rate.

def qtz_dissolution_rate(T,S,omega,aHplus,aOHminus):
    
    # parameters for neutral mechanism
    k_nu=6.4e-14
    Ea_nu=77*1000.0 # to J

    #Acid mechanism depends on H+
    k_H=0.0 # switch off acid mechanism
    Ea_H=0.0 # to J
    n_H=0.0 # exponent for H+ activity
    
    
    #parameters for base mechanism depends on OH-
    k_OH=1.9e-10
    Ea_OH=80.0*1000.0 # to J
    n_OH=0.34 # exponent for H+ activity    

    # other parameters
    R=8.314462618

    # calculate in parts
    T_nu_term = np.exp( (-Ea_nu)/R*(1.0/T-1.0/298.15) )
    neutral_term=k_nu*T_nu_term*S*(1.0-omega) 
    T_OH_term=np.exp( (1.0/T-1.0/298.15)*(-1.0)*Ea_OH/R )*pow(aOHminus,n_OH)
    base_term= k_OH*T_OH_term*S*(1.0-omega) # or pH like in the other scripts
    T_H_term=np.exp( (1.0/T-1.0/298.15)*(-1.0)*Ea_H/R )*pow(aHplus,(n_H))
    acid_term= k_H*T_H_term*S*(1.0-omega) # or pH like in the other scripts
    
    qtz_dissolution_rate = ( neutral_term + base_term + acid_term)
    
    return qtz_dissolution_rate 

In [ ]:
# example of how a function works 
# secific surface area * 26.7 mol qtz, << undersaturated, basic pH 
print('qtz dissolution rate', qtz_dissolution_rate(298.15, 9.0e-2*26.7, 8.0e-06, 5.9e-14, 0.17), 'mol/s')

## Set global parameters, run test equilibration

In [ ]:
print('Script for aggregate dissolution with gems kernel')

# Initial equilibration
# take some parameters as read !
T = xgEngine.temperature()
P = xgEngine.pressure()
b = xgEngine.elementAmounts().copy()

# now set upper lower limits for quartz:
xgEngine.setSpeciesLowerLimit("Qtz",26.679)
#xgEngine.setSpeciesUpperLimit("hydrotalcite",1e-9)

xgEngine.setColdStart()
code = xgEngine.equilibrate(T, P, b)
if code != 2 and code != 6:
            print("WARNING: Problem with GEMS output code at the beginning...... ")
            sys.exit()

# we storr some common used indexes in own variables 
index_qtz=xgEngine.indexSpecies('Qtz')
index_OHminus=xgEngine.indexSpecies('OH-')
index_Hplus=xgEngine.indexSpecies('H+')
 
print('dcomp index for Qtz', index_qtz,'OH-', index_OHminus,' H+ ',index_Hplus) 

## Dissolution time loop

In [ ]:
# use SI units
year_to_seconds=365.25*24.0*3600.0

# Temperature in Kelvin
T=25.0+273.15

# definitions for time discretization
tend=5000.0 *year_to_seconds # unit seconds!!!
delta_t=1.0 *year_to_seconds
time = 0.0 # start

time=time+delta_t # only for first time step
code = 2

# while loop
while time <= tend :
    print('\rcalculate for time = '+str(time/year_to_seconds)+' years', 'dt ', delta_t/year_to_seconds, 'code', code, end='')    

    qtz_amount=xgEngine.speciesAmount(index_qtz) # curent amount 
    omega = pow(10,xgEngine.phaseSatIndex(phase_indexes['Quartz'])) # omega of qtz
    
    aHplus = np.exp(xgEngine.lnActivities()[index_Hplus]) # activity of H+
    aOHminus = np.exp(xgEngine.lnActivities()[index_OHminus]) # activity of OH-

    #   we assume that we have much more aggregates than we need to reach equilibrium with Quartz, 
    #   i.e. rounded grains change surface are not completely dissolved  
    SurfaceArea=qtz_amount*9.065e-2 # specific surface area m2/mol Table 4.8 of NAB 18-05 1.5 mm spherical grains or so
    
    # we call the rate function
    drate = qtz_dissolution_rate(T,SurfaceArea,omega,aHplus,aOHminus) 

    # we calculate the low_limit dll_ for qtz
    low_limit=qtz_amount-drate*delta_t
    
    # lower limit should be never below zero
    if (low_limit < 0.0):
        low_limit=0.0

    if (np.log10(omega) < 0.0) :    # only change lower limit if dissolution happens
        xgEngine.setSpeciesLowerLimit("Qtz",low_limit)
    
    # we run GEMS            
    xgEngine.setColdStart()
    code = xgEngine.equilibrate(T, P, b)

    # check if GEMS calculation was succefful, if not try to imporve numerics and reequilibrate
    if not (code == 2 or code == 6):  # 2 and 6 are  good solution ...3 is maybe good solution due to divergence problems
        print('t , dt ',time,' ',delta_t,' rescaled limits and run again\n')
        xgEngine.setSpeciesLowerLimit("Qtz",low_limit*0.99999999999999)
        code =xgEngine.reequilibrate(False)
        if not (code == 2):
            print("Error: Problem with GEMS output code during equilibration table! at time returned code ",time,code)
            print("for detailed error diagnostics look into ipmlog.txt")
            print('t , dt ',time,' ',delta_t,' rate: ',drate,' SurfaceArea: ',SurfaceArea,' amount per dt: ',drate*dt)
            print("bulk composition",xgEngine.elementAmounts())
            print("dll",xgEngine.speciesLowerLimits())
            time= 2*tend
            outflag = False
            break

    ## store results
    # copy from leaching example, change cycle to time, add rate
    aq_volume = xgEngine.phaseVolume(phase_indexes['aq_gen']) * 1000  # m³ to L
    aq_mass = xgEngine.phaseMass(phase_indexes['aq_gen'])  # kg
    aq_elements = xgEngine.elementAmountsInPhase(phase_indexes['aq_gen'])
    CSH_elements = xgEngine.elementAmountsInPhase(phase_indexes['CSHQ'])
    
    # properties
    properties_table['time'].append(time/year_to_seconds)
    properties_table['Ca/Si in CSH'].append(CSH_elements[element_indexes['Ca']]/CSH_elements[element_indexes['Si']])
    properties_table['pH'].append(xgEngine.pH())
    properties_table['gems_code'].append(code)
    properties_table['rate'].append(drate)

    # Aqueous species concentrations
    for element in output_aqueous_elements:
        index = element_indexes[element]
        aqueous_table[element].append(1000*aq_elements[index] / aq_volume)

    # Solid phase volumes
    for phase, ndx_list in grouped_output_phase_indexes.items():
        vol = sum(xgEngine.phaseVolume(ndx) for ndx in ndx_list)
        solids_volume_table[phase].append(vol * 1e6)  # m³ to cm³

    # increase the time
    time=time+delta_t
    # delta_t=delta_t*1.01
    if time/year_to_seconds > 1000.0 : # accelerate time after 1000 years
        delta_t = delta_t*1.01

print("\neverything finished")

In [ ]:
# copy from leaching example and change cycle to time
# plot the results
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 15})

# Create plot
fig, ax1 = plt.subplots(figsize=(7, 5))

for elem in output_aqueous_elements:
    ax1.plot(properties_table['time'], aqueous_table[elem], label=elem)

ax1.set_xlabel('time (years)')
ax1.set_ylabel('total conc. in solution (mmol/l)')
ax1.set_title('quartz dissolution')
ax1.grid(True)
#ax1.set_xscale('log')

ax2 = ax1.twinx()
ax2.plot(properties_table['time'], properties_table['pH'], label='pH', color='black', linestyle='--')
ax2.set_ylabel('pH')


lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# copy from the leaching example and change cycle to time, 
# ax.set_ylim(600, 1000) as example of zooming in
# ax.set_xscale('log')
# Assume solid_data_volumes is a dictionary filled with lists of equal length
# Extract the x-axis (cycle) and the y-values (each mineral phase)
x = properties_table['time']

# Y-values in correct order
y_values = [solids_volume_table[phase] for phase in solids_volume_table]

# Labels for legend
labels = [phase for phase in solids_volume_table]  # You can make this prettier manually if needed

# Color palette (optional): provide your own or use matplotlib defaults
colors = plt.cm.tab20c(range(len(solids_volume_table)))  # or a custom list

# Optional hatches
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*', '//', 'xx', '\\\\', '||', '++', 'oo', '**', '..']

# Create the stacked area plot
fig, ax = plt.subplots(figsize=(12, 6))
stacks = ax.stackplot(x, *y_values, labels=labels, colors=colors, alpha=0.8)

# Apply hatching
for stack, hatch in zip(stacks, hatches):
    stack.set_hatch(hatch)

# Axis labels and limits
ax.set_xlabel('Time (years)')
ax.set_ylabel('Phase volume')
ax.set_title('Solid Phase Volume Evolution During quartz dissolution')
ax.set_ylim(600, 1000)
#ax.set_xscale('log')


# Create a second y-axis for pH
ax2 = ax.twinx()
ax2.plot(x, properties_table['Ca/Si in CSH'], label='Ca/Si in CSH', color='black', linestyle='--')
ax2.set_ylabel('Ca/Si in CSH')

# Combine legends from both axes
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right',  bbox_to_anchor=(1.35, 1.0), fontsize=10)

plt.tight_layout()
plt.show()